In [1]:
import pandas as pd
import numpy as np
import sys
import re
import pymysql

import hgtk
from hangul_utils import join_jamos

"""
* 가타카나 자동화V3.3 *

두레이 프로젝트링크 : https://redtable.dooray.com/project/important/3214559310182663970?page=1
구글 스프레드시트 : https://docs.google.com/spreadsheets/d/1hhD6tqWJeKd7H2bvxBVZ2aYxZQIxczrLoludQ8woe7w/edit#gid=0

업데이트일시 : 2022.12.29
업데이트내용 : 
    1) 모듈 import 순서 변경 (표준 라이브러리 모듈 > 3rd Party 모듈 > 직접 만든 모듈 순)
    2) EAFP(Easier to Ask Forgiveness than Permission) 형식으로 변경
    3) 코드 최적화 진행
    4) 사이시옷 뒤에 ‘이’ 음이 결합되는 경우에는 [ㄴㄴ]으로 발음 적용
업데이트결과 : 
    1) 성능테스트 샘플 1만개 진행시 속도 증가
        V3.2 : 47.14267 sec
        V3.3 : 7.01863 sec
    2) 코드단축
        V3.2 : 1237줄
        V3.3 : 834줄
        
담당자 : Hob
"""

"""
개발DB dictionary에 있는 테이블을 사전으로 활용하기
"""
db_host = 'db-7ma06.pub-cdb.ntruss.com'
db_user = 'redtable_openapi'
db_password = 'fpemxpdlqmf5491!@#'
db_port = 3306
db_name = 'dictionary'

conn = pymysql.connect(host=db_host, user=db_user, password=db_password, port=db_port, db=db_name, charset='utf8')
cursor = conn.cursor()

"""
#1 kata_JAMO : 국어에서 자모음이 가타카나로 변환 되는 사전
#2 kata_JONG : 국어에서 자모음이 가타카나로 변환 되는 사전
#3 kata_except_JAMO : 국어에서 자모음이 기본 규칙이 아닌 예외규칙으로 적용되어 가타카나로 변환 되는 사전
#4 kata_except_JONG : 국어에서 받침이 기본 규칙이 아닌 예외규칙으로 적용되어 가타카나로 변환 되는 사전
#5 kata_standard_pro : 가타카나 변환을 위한 표준발음법 규칙적용표
#6 kata_dic_idiom : 가타카나 외래어관용어 사전
"""
sql = "SELECT * FROM dictionary.kata_JAMO"
cursor.execute(sql)
kata_JAMO = cursor.fetchall()
kata_JAMO = pd.DataFrame(kata_JAMO, columns=["id", "hangeul_JAMO", "kata_JAMO"])
JAMO_dic = kata_JAMO["hangeul_JAMO"].values

sql = "SELECT * FROM dictionary.kata_JONG"
cursor.execute(sql)
kata_JONG = cursor.fetchall()
kata_JONG = pd.DataFrame(kata_JONG, columns=["id", "hangeul_JONG", "hangeul_standardPro", "represent", "kata_JONG"])
JONG_dic = kata_JONG['hangeul_JONG'].values

sql = "SELECT * FROM dictionary.kata_except_JAMO"
cursor.execute(sql)
kata_except_JAMO = cursor.fetchall()
kata_except_JAMO = pd.DataFrame(kata_except_JAMO, columns=["id", "hangeul_except_JAMO", "kata_except_JAMO"])
except_JAMO_dic = kata_except_JAMO['hangeul_except_JAMO'].values

sql = "SELECT * FROM dictionary.kata_except_JONG"
cursor.execute(sql)
kata_except_JONG = cursor.fetchall()
kata_except_JONG = pd.DataFrame(kata_except_JONG, columns=["id", "hangeul_except_JONG", "kata_except_JAMO"])
except_JONG_dic = kata_except_JONG['hangeul_except_JONG'].values
kata_except_JONG_dic = kata_except_JONG['kata_except_JAMO'].values

sql = "SELECT * FROM dictionary.kata_standard_pro"
cursor.execute(sql)
kata_standard_pro = cursor.fetchall()
kata_standard_pro = pd.DataFrame(kata_standard_pro, columns=["id", "standard_pro_hangeul", "standard_pro_JA", "standard_pro_MO_JONG", "standard_pro_JONG"])
standard_pro_dic = kata_standard_pro['standard_pro_hangeul'].values

sql = "select * from dictionary.kata_dic_idiom"
cursor.execute(sql)
kata_dic_idiom = cursor.fetchall()
kata_dic_idiom = pd.DataFrame(kata_dic_idiom, columns=["id", "article", "kr_word", "jp_word", "kr_word_len"])
idiom_arr = kata_dic_idiom.to_numpy()

cursor.close()
conn.close()

def isHangul(text):
    """
    text가 한글(자음,모음,단모음) 여부를 확인
    결과값은 True / False 로 나옴
    입    력 : ㄱa1
    결    과 : True
    """
    try:
        hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', text))
        return hanCount > 0
    except:
        return False
    
def combine(text):
    """
    한 단어의 한글이 입력되면 종성만 분리된다
    입    력 : 안
    결    과 : 아ㄴ
    """
    try:
        if isHangul(text) == True and len(text) == 1:
            korea_text = text
            CHO   = hgtk.letter.decompose(korea_text)[0]
            JOONG = hgtk.letter.decompose(korea_text)[1]
            JONG  = hgtk.letter.decompose(korea_text)[2]

            combine_text = hgtk.letter.compose(CHO, JOONG, '')
            result = (combine_text, JONG)
            result = str(result).replace('(','').replace(')','').replace("'",'').replace(',','').replace(' ','')
            return result
        else:
            return text
    except:
        print("combine 함수 에러발생")
        return text
    
def segment(text):
    """
    한글이 입력되면 종성만 분리된다
    입    력 : 안녕육지사람
    결    과 : 아ㄴ녀ㅇ유ㄱ지사라ㅁ
    """
    try:
        result_text = ""
        for i in range(len(text)):
            if isHangul(text[i]) == True:
                Hangul_text = text[i]
                for j in range(len(Hangul_text)):
                    result_text = "".join([result_text, combine(Hangul_text[j])])
                    continue
            else:
                result_text = "".join([result_text, text[i]])
        return result_text
    
    except:
        print("segment 함수 에러발생")
        return text

def JAMO_exception(text):
    """
    자모음 예외처리1 :‘ㄱ’,‘ㄷ’,‘ㅂ’,‘ㅈ’이 유성음 사이에 올 경우, 유성음: ㄴ, ㄹ, ㅁ, ㅇ
    입    력 : 여의도
    기존결과 : ヨイト
    예외결과 : ヨイド
    """
    result_text = ""
    for i in range(len(text)):
        if text[i] in except_JAMO_dic and i > 0:
            result_text = "".join([result_text, kata_except_JAMO['kata_except_JAMO'][np.where(text[i]==except_JAMO_dic)[0][0]]])
            continue
        else:
            result_text = "".join([result_text, text[i]])
            continue
    return result_text

def JAMO_exception2(text):
    """
    자모음 예외처리2 :‘ㅢ’가 첫음절 다음에 쓰일 경우 "의"가 첫음절을 제외하고 나올때 "イ"로 발음, 
                        단어의 마지막에 올경우에는 "エ"로 발음
    전제조건 : 띄어쓰기 없는 단어
    입    력 : 나의 여의도
    결    과 : 나エ 여イ도
    """
    result_text = ""
    for i in range(len(text)):
        if "의" == text[i] and i > 0 and i != (len(text) - 1):
            result_text = "".join([result_text, "イ"])
            continue
        elif "의" == text[len(text) - 1] and i == (len(text) - 1):
            result_text = "".join([result_text, "エ"])
            continue
        else:
            result_text = "".join([result_text, text[i]])
            continue
    return result_text

def JONG_exception(text):
    """
    받침 예외처리 :
        ‘ㄱ, ㅋ, ㄲ, ㄳ, ㄺ’ 받침 다음에 ‘ㄱ, ㅋ, ㄲ’ 초성이 올 경우 ッ(촉음)
        ‘ㅂ, ㅍ, ㅃ, ㄼ, ㄿ, ㅄ’ 받침 다음에 ‘ㅂ, ㅍ, ㅃ’ 초성이 올 경우 ッ(촉음)
        ‘ㅁ, ㄻ’ 받침 다음에 ‘ㅁ, ㅂ, ㅃ, ㅍ’ 초성이 올 경우 ン
    입    력 : 겨ㅇ보ㄱ구ㅇ
    결    과 : 겨ㅇ보ッ구ㅇ
    """
    for i in range(len(except_JONG_dic)):
        if except_JONG_dic[i] in text:
            text = text.replace(except_JONG_dic[i], kata_except_JONG_dic[i])
            continue
    return text

def kata_idiom_replace(text):
    """
    외래어 관용어
    입력 : 투썸플레이스
    결과 : ツーサムプレイス
    """
    for i in idiom_arr:
        if len(i[2]) <= len(text):
            if i[2] in text:
                text = text.replace(i[2],i[3])
    return text

def katakanize(text):
    """
    가타카나 자모음 규칙에 따라 한글 -> 가타카나로 변환되는 함수
    전제조건 : 띄어쓰기 없는 단어
    결    과 : 가타카나(str)
    """
    result_text = ""
    for i in range(len(text)):
        letter = text[i]
        if letter in JAMO_dic:
            result_text = "".join([result_text, kata_JAMO['kata_JAMO'][np.where(letter==JAMO_dic)[0][0]]])
            continue
        elif letter in JONG_dic:
            result_text = "".join([result_text, kata_JONG['kata_JONG'][np.where(letter==JONG_dic)[0][0]]])
            continue
        else:
            result_text = "".join([result_text, letter])
            continue

    return result_text

def standardPro(text):
    """
    표준발음법 적용하는 함수
    """
    result_text = ""
    for i in range(len(text)):
        try:
            letter = hgtk.letter.decompose(text[i])
            if i == 0:
                if letter[0] in standard_pro_dic:
                    result_text = "".join([result_text, kata_standard_pro['standard_pro_JA'][np.where(letter[0]==standard_pro_dic)[0][0]]])
                if letter[1] in standard_pro_dic:
                    result_text = "".join([result_text, kata_standard_pro['standard_pro_JA'][np.where(letter[1]==standard_pro_dic)[0][0]]])
                if letter[2] in standard_pro_dic:
                    # 글자 수가 한 글자일때
                    if i == (len(text)-1):
                        result_text = "".join([result_text, kata_standard_pro['standard_pro_JONG'][np.where(letter[2]==standard_pro_dic)[0][0]]])

                    # 글자 수가 두 글자 이상이고 가장 앞단어
                    elif i != (len(text)-1):
                        try:
                            letter2 = hgtk.letter.decompose(text[i+1])
                            
                            # 사이시옷 뒤에 ‘이’ 음이 결합되는 경우에는 [ㄴㄴ]으로 발음한다
                            # 깻잎 [깨십] -> [깬닙]
                            if letter[2] == 'ㅅ' and letter2[0] == 'ㅇ':
                                result_text = "".join([result_text, 'ㄴ'])
                            
                            # 제10항　겹받침 'ㄳ', 'ㄵ', 'ㄼ, ㄽ, ㄾ', 'ㅄ'은 어말 또는 자음 앞에서 각각 [ㄱ, ㄴ, ㄹ, ㅂ]으로 발음한다.
                            # 다만, '밟-'은 자음 앞에서 [밥]으로 발음하고, '넓-'은 다음과 같은 경우에 [넙]으로 발음한다.
                            elif letter[0] == 'ㅂ' and letter[1] == 'ㅏ' and letter[2] == 'ㄼ' and letter2[0] != 'ㄴ' and letter2[0] in standard_pro_dic:
                                result_text = "".join([result_text, 'ㅂ'])
                            elif letter[0] == 'ㅂ' and letter[1] == 'ㅏ' and letter[2] == 'ㄼ' and letter2[0] == 'ㄴ':
                                result_text = "".join([result_text, 'ㅁ' ])

                            # 제10항 다만 과 제25항 만남
                            # 넓은, 넓게, 넓다, 넓이, 넓어, 넓고, 넓으(면) 제외

                            # 제25항　어간 받침 'ㄼ, ㄾ' 뒤에 결합되는 어미의 첫소리 'ㄱ, ㄷ, ㅅ, ㅈ'은 된소리로 발음한다.
                            elif letter[0] == 'ㄴ' and letter[1] == 'ㅓ' and letter[2] == 'ㄼ':
                                if (letter2[0] == 'ㅇ' and letter2[1] == 'ㅡ' and letter2[2] == 'ㄴ') \
                                or (letter2[0] == 'ㄱ' and letter2[1] == 'ㅔ' and letter2[2] not in standard_pro_dic) \
                                or (letter2[0] == 'ㄷ' and letter2[1] == 'ㅏ' and letter2[2] not in standard_pro_dic) \
                                or (letter2[0] == 'ㅇ' and letter2[1] == 'ㅣ' and letter2[2] not in standard_pro_dic) \
                                or (letter2[0] == 'ㅇ' and letter2[1] == 'ㅓ' and letter2[2] not in standard_pro_dic) \
                                or (letter2[0] == 'ㄱ' and letter2[1] == 'ㅗ' and letter2[2] not in standard_pro_dic) \
                                or (letter2[0] == 'ㅇ' and letter2[1] == 'ㅡ' and letter2[2] not in standard_pro_dic):
                                    result_text = "".join([result_text, 'ㄹ'])
                                else:
                                    result_text = "".join([result_text, 'ㅂ'])

                            # 제11항　겹받침 'ㄺ, ㄻ, ㄿ'은 어말 또는 자음 앞에서 각각 [ㄱ, ㅁ, ㅂ]으로 발음한다.
                            # 제11항 다만 예외 중 흙과[흑꽈]에 관련된 내용
                            elif letter[2] == 'ㄺ' and letter2[0] == 'ㄱ' and letter2[1] in 'ㅘ,ㅟ,ㅙ,ㅝ,ㅞ':
                                result_text = "".join([result_text, 'ㄱ' ])

                            # 다만, 용언의 어간 말음 'ㄺ'은 'ㄱ' 앞에서 [ㄹ]로 발음한다.
                            elif letter[2] == 'ㄺ' and letter2[0] == 'ㄱ' and (letter2[1] == 'ㅔ' or letter2[1] == 'ㅗ' or letter2[1] == 'ㅓ') \
                            and letter2[2] not in standard_pro_dic:
                                result_text = "".join([result_text, 'ㄹ'])

                            # 제12항 받침 'ㅎ'의 발음은 다음과 같다.
                            #    1. 'ㅎ(ㄶ, ㅀ)' 뒤에 'ㄱ, ㄷ, ㅈ'이 결합되는 경우에는, 뒤 음절 첫소리와 합쳐서 [ㅋ, ㅌ, ㅊ]으로 발음한다.
                            # 'ㅎ'
                            elif letter[2] == 'ㅎ' and (letter2[0] == 'ㄱ' or letter2[0] == 'ㄷ' or letter2[0] == 'ㅈ'):
                                result_text = "".join([result_text, '▒'])
                            # 'ㄶ'
                            elif letter[2] == 'ㄶ' and (letter2[0] == 'ㄱ' or letter2[0] == 'ㄷ' or letter2[0] == 'ㅈ'):
                                result_text = "".join([result_text, 'ㄴ'])
                            # 'ㅀ'
                            elif letter[2] == 'ㅀ' and (letter2[0] == 'ㄱ' or letter2[0] == 'ㄷ' or letter2[0] == 'ㅈ'):
                                result_text = "".join([result_text, 'ㄹ'])
                            #     [붙임 1] 받침 'ㄱ(ㄺ),ㄷ,ㅂ(ㄼ),ㅈ(ㄵ)'이 뒤 음절 첫소리 'ㅎ'과 결합되는 경우에도,
                            #              역시 두 소리를 합쳐서 [ㅋ,ㅌ,ㅍ,ㅊ]으로 발음한다.
                            # 'ㄱ,ㄷ,ㅂ,ㅈ'
                            elif (letter[2] == 'ㄱ' or letter[2] == 'ㄷ' or letter[2] == 'ㅂ' or letter[2] == 'ㅈ') and letter2[0] == 'ㅎ':
                                result_text = "".join([result_text, '▒'])
                            # 'ㄺ,ㄼ,ㄵ'
                            elif letter[2] == 'ㄺ' and letter2[0] == 'ㅎ':
                                result_text = "".join([result_text, 'ㄱ'])
                            elif letter[2] == 'ㄼ' and letter2[0] == 'ㅎ':
                                result_text = "".join([result_text, 'ㄹ'])
                            elif letter[2] == 'ㄵ' and letter2[0] == 'ㅎ':
                                result_text = "".join([result_text, 'ㄴ'])
                            #     [붙임 2] 규정에 따라 'ㄷ'으로 발음되는 'ㅅ, ㅈ, ㅊ, ㅌ'의 경우에는 이에 준한다.
                            # 'ㅅ,ㅈ,ㅊ,ㅌ'
                            elif (letter[2] == 'ㅅ' or letter[2] == 'ㅈ' or letter[2] == 'ㅊ' or letter[2] == 'ㅌ') and letter2[0] == 'ㅎ':
                                result_text = "".join([result_text, '▒'])

                            #    2. 'ㅎ(ㄶ, ㅀ)' 뒤에 'ㅅ'이 결합되는 경우에는, 'ㅅ'을 [ㅆ]으로 발음한다.
                            elif letter[2] == 'ㅎ' and letter2[0] == 'ㅅ':
                                result_text = "".join([result_text, '▒'])
                            elif letter[2] == 'ㄶ' and letter2[0] == 'ㅅ':
                                result_text = "".join([result_text, 'ㄴ'])
                            elif letter[2] == 'ㅀ' and letter2[0] == 'ㅅ':
                                result_text = "".join([result_text, 'ㄹ'])

                            #    3. 'ㅎ' 뒤에 'ㄴ'이 결합되는 경우에는, [ㄴ]으로 발음한다.
                            elif letter[2] == 'ㅎ' and letter2[0] == 'ㄴ':
                                result_text = "".join([result_text, 'ㄴ'])
                            #     [붙임] 'ㄶ, ㅀ' 뒤에 'ㄴ'이 결합되는 경우에는, 'ㅎ'을 발음하지 않는다.
                            # 'ㄶ, ㅀ'
                            elif letter[2] == 'ㄶ' and letter2[0] == 'ㄴ':
                                result_text = "".join([result_text, 'ㄴ'])
                            elif letter[2] == 'ㅀ' and letter2[0] == 'ㄴ':
                                result_text = "".join([result_text, 'ㄹ'])

                            #    4. 'ㅎ(ㄶ, ㅀ)' 뒤에 모음으로 시작된 어미나 접미사가 결합되는 경우에는, 'ㅎ'을 발음하지 않는다.
                            elif (letter[2] == 'ㅎ' or letter[2] == 'ㄶ' or letter[2] == 'ㅀ') and letter2[0] == 'ㅇ':
                                result_text = "".join([result_text, '▒'])

                            # 제17항　받침 'ㄷ, ㅌ(ㄾ)'이 조사나 접미사의 모음 'ㅣ'와 결합되는 경우에는, [ㅈ, ㅊ]으로 바꾸어서 뒤 음절 첫소리로 옮겨 발음한다.
                            elif letter2[1] == 'l' and letter2[0] == 'ㅇ' and letter2[2] not in standard_pro_dic and letter[2] == 'ㄾ':
                                result_text = "".join([result_text, 'ㄹ'])
                            elif letter2[1] == 'l' and letter2[0] == 'ㅇ' and letter2[2] not in standard_pro_dic and (letter[2] == 'ㄷ' or letter[2] == 'ㅌ'):
                                result_text = "".join([result_text, '▒'])
                                    
                            #    [붙임] 'ㄷ' 뒤에 접미사 '히'가 결합되어 '티'를 이루는 것은 [치]로 발음한다.
                            elif letter2[1] == 'l' and letter[2] == 'ㄷ' and letter2[0] == 'ㅎ' and letter2[2] not in standard_pro_dic:
                                result_text = "".join([result_text, '▒'])

                            # 제13항 홑받침이나 쌍받침이 모음으로 시작된 조사나 어미, 접미사와 결합되는 경우에는, 제 음가대로 뒤 음절 첫소리로 옮겨 발음한다.
                            elif letter2[0] == 'ㅇ' and letter[2] in standard_pro_dic and letter[2] not in 'ㄳ,ㄵ,ㄼ,ㄽ,ㄾ,ㅄ,ㄺ,ㄻ,ㄿ,ㅀ,ㄶ,ㅃ,ㅉ,ㄸ':
                                # 종성'ㅇ', 초성'ㅇ'인 경우 종성'ㅇ'이 사라집니다. 예) 능이->느이 , 장어->자어 , 영양 -> 여양
                                if letter[2] == 'ㅇ' and letter2[0] == 'ㅇ':
                                    result_text = "".join([result_text, 'ㅇ'])
                                else:
                                    result_text = "".join([result_text, '▒'])

                            # 제14항　겹받침이 모음으로 시작된 조사나 어미, 접미사와 결합되는 경우에는, 뒤엣것만을 뒤 음절 첫소리로 옮겨 발음한다.
                            # (이 경우, 'ㅅ'은 된소리로 발음함.)    
                            elif letter2[0] == 'ㅇ' and letter[2] in 'ㄳ,ㄵ,ㄼ,ㄽ,ㄾ,ㅄ,ㄺ,ㄻ,ㄿ,ㅀ,ㄶ':
                                if letter[2] == 'ㄳ' and letter2[0] == 'ㅇ':
                                    result_text = "".join([result_text, 'ㄱ'])
                                elif (letter[2] == 'ㄵ' or letter[2] == 'ㄶ') and letter2[0] == 'ㅇ':
                                    result_text = "".join([result_text, 'ㄴ'])
                                elif (letter[2] == 'ㄼ' or letter[2] == 'ㄽ' or letter[2] == 'ㄾ' or letter[2] == 'ㄺ' \
                                    or letter[2] == 'ㄻ' or letter[2] == 'ㄿ' or letter[2] == 'ㅀ') and letter2[0] == 'ㅇ':
                                    result_text = "".join([result_text, 'ㄹ'])
                                elif letter[2] == 'ㅄ' and letter2[0] == 'ㅇ':
                                    result_text = "".join([result_text, 'ㅂ'])
                                # 종성'ㅇ', 초성'ㅇ'인 경우 종성'ㅇ'이 사라집니다. 예) 능이->느이 , 장어->자어 , 영양 -> 여양
                                elif letter[2] == 'ㅇ' and letter2[0] == 'ㅇ':
                                    result_text = "".join([result_text, 'ㅇ'])
                                else:
                                    result_text = "".join([result_text, '▒'])

                            # 제18항 받침 'ㄱ(ㄲ, ㅋ, ㄳ, ㄺ), ㄷ(ㅅ, ㅆ, ㅈ, ㅊ, ㅌ, ㅎ), ㅂ(ㅍ, ㄼ, ㄿ, ㅄ)'은 'ㄴ, ㅁ' 앞에서 [ㅇ, ㄴ, ㅁ]으로 발음한다.
                            # 'ㄱ,ㄲ,ㅋ,ㄳ,ㄺ' -> 'ㅇ'
                            elif (letter[2] == 'ㄱ' or letter[2] == 'ㄲ' or letter[2] == 'ㅋ' or letter[2] == 'ㄳ' or letter[2] == 'ㄺ') \
                            and (letter2[0] == 'ㄴ' or letter2[0] == 'ㅁ'):
                                result_text = "".join([result_text, 'ㅇ'])
                            # 'ㄷ.ㅅ.ㅆ.ㅈ.ㅊ.ㅌ.ㅎ' -> 'ㄴ'
                            elif (letter[2] == 'ㄷ' or letter[2] == 'ㅅ' or letter[2] == 'ㅆ' or letter[2] == 'ㅈ' \
                                  or letter[2] == 'ㅊ' or letter[2] == 'ㅌ' or letter[2] == 'ㅎ') \
                            and (letter2[0] == 'ㄴ' or letter2[0] == 'ㅁ'):
                                result_text = "".join([result_text, 'ㄴ'])
                            # 'ㅂ,ㅍ,ㄼ,ㄿ,ㅄ' -> 'ㅁ'
                            elif (letter[2] == 'ㅂ' or letter[2] == 'ㅍ' or letter[2] == 'ㄼ' or letter[2] == 'ㄿ' or letter[2] == 'ㅄ') \
                            and (letter2[0] == 'ㄴ' or letter2[0] == 'ㅁ'):
                                result_text = "".join([result_text, 'ㅁ'])

                            # 제19항　받침 'ㅁ, ㅇ' 뒤에 연결되는 'ㄹ'은 [ㄴ]으로 발음한다.
                            elif letter[2] == 'ㅁ' and letter2[0] == 'ㄹ':
                                result_text = "".join([result_text, 'ㅁ'])
                            elif letter[2] == 'ㅇ' and letter2[0] == 'ㄹ':
                                result_text = "".join([result_text, 'ㅇ'])

                            # 제19항 [붙임] 받침 'ㄱ, ㅂ' 뒤에 연결되는 'ㄹ'도 [ㄴ]으로 발음한다.
                            elif letter[2] == 'ㄱ' and letter2[0] == 'ㄹ':
                                result_text = "".join([result_text, 'ㅇ'])
                            elif letter[2] == 'ㅂ' and letter2[0] == 'ㄹ':
                                result_text = "".join([result_text, 'ㅁ'])

                            # 제23항 받침 'ㄱ(ㄲ, ㅋ, ㄳ, ㄺ), ㄷ(ㅅ, ㅆ, ㅈ, ㅊ, ㅌ), ㅂ(ㅍ, ㄼ, ㄿ, ㅄ)' 뒤에 연결되는 'ㄱ, ㄷ, ㅂ, ㅅ, ㅈ'은 된소리로 발음한다.
                            # => 종성은 변화규칙X 초성만 변화규칙 적용
                            elif letter2[0] != 'ㅇ' and letter[2] in standard_pro_dic:
                                result_text = "".join([result_text, kata_standard_pro['standard_pro_JONG'][np.where(letter[2]==standard_pro_dic)[0][0]]])
                            
                        except:
                            result_text = "".join([result_text, kata_standard_pro['standard_pro_JONG'][np.where(letter[2]==standard_pro_dic)[0][0]]])

                    else:
                        result_text = "".join([result_text, kata_standard_pro['standard_pro_MO_JONG'][np.where(letter[2]==standard_pro_dic)[0][0]]])

            else:
                letter3 = hgtk.letter.decompose(text[i-1])
                if letter[0] in standard_pro_dic:
                    # 국회의사당 오류를 위한 코드 ex) 구쾨ㅢ사당 -> 구쾨의사당
                    if letter[0] == 'ㅇ' and letter3[2] not in standard_pro_dic:
                        result_text = "".join([result_text, 'ㅇ'])

                    # 제 12항    
                    #    4. 'ㅎ(ㄶ, ㅀ)' 뒤에 모음으로 시작된 어미나 접미사가 결합되는 경우에는, 'ㅎ'을 발음하지 않는다.
                    elif letter[0] == 'ㅇ' and letter3[2] == 'ㅎ' and letter[1] in standard_pro_dic:
                        result_text = "".join([result_text, 'ㅇ'])
                    elif letter[0] == 'ㅇ' and letter3[2] == 'ㄶ' and letter[1] in standard_pro_dic:
                        result_text = "".join([result_text, 'ㄴ'])
                    elif letter[0] == 'ㅇ' and letter3[2] == 'ㅀ' and letter[1] in standard_pro_dic:
                        result_text = "".join([result_text, 'ㄹ'])

                    # 제17항　받침 'ㄷ, ㅌ(ㄾ)'이 조사나 접미사의 모음 'ㅣ'와 결합되는 경우에는, [ㅈ, ㅊ]으로 바꾸어서 뒤 음절 첫소리로 옮겨 발음한다.
                    elif letter[1] == 'ㅣ' and letter3[2] in standard_pro_dic and letter[0] == 'ㅇ' and letter3[2] == 'ㄷ':
                        result_text = "".join([result_text, 'ㅈ'])
                    elif letter[1] == 'ㅣ' and letter3[2] in standard_pro_dic and letter[0] == 'ㅇ' \
                    and (letter3[2] == 'ㅌ' or letter3[2] == 'ㄾ'):
                        result_text = "".join([result_text, 'ㅊ'])

                    # 제13항 홑받침이나 쌍받침이 모음으로 시작된 조사나 어미, 접미사와 결합되는 경우에는, 제 음가대로 뒤 음절 첫소리로 옮겨 발음한다.
                    elif letter[0] == 'ㅇ' and letter3[2] in standard_pro_dic and letter3[2] not in 'ㄳ,ㄵ,ㄼ,ㄽ,ㄾ,ㅄ,ㄺ,ㄻ,ㄿ,ㅀ,ㄶ,ㅃ,ㅉ,ㄸ,ㅎ' \
                    and letter3[2] not in 'ㅅ':
                        result_text = "".join([result_text, letter3[2]])
                    
                    # 사이시옷 뒤에 ‘이’ 음이 결합되는 경우에는 [ㄴㄴ]으로 발음한다
                    # 깻잎 내용추가로 인한 변경
                    elif letter[0] == 'ㅇ' and letter3[2] == 'ㅅ':
                        result_text = "".join([result_text, 'ㄴ'])
                    
                    # 제14항　겹받침이 모음으로 시작된 조사나 어미, 접미사와 결합되는 경우에는, 뒤엣것만을 뒤 음절 첫소리로 옮겨 발음한다.
                    # (이 경우, 'ㅅ'은 된소리로 발음함.)
                    elif letter[0] == 'ㅇ' and letter3[2] in 'ㄳ,ㄵ,ㄼ,ㄽ,ㄾ,ㅄ,ㄺ,ㄻ,ㄿ,ㅀ,ㄶ':
                        if (letter3[2] == 'ㄳ' or letter3[2] == 'ㄽ' or letter3[2] == 'ㅄ'):
                            result_text = "".join([result_text, 'ㅆ'])
                        elif letter3[2] == 'ㄵ':
                            result_text = "".join([result_text, 'ㅈ'])
                        elif letter3[2] == 'ㄼ':
                            result_text = "".join([result_text, 'ㅂ'])
                        elif letter3[2] == 'ㄾ':
                            result_text = "".join([result_text, 'ㅌ'])
                        elif letter3[2] == 'ㄺ':
                            result_text = "".join([result_text, 'ㄱ'])
                        elif letter3[2] == 'ㄻ':
                            result_text = "".join([result_text, 'ㅁ'])
                        elif letter3[2] == 'ㄿ':
                            result_text = "".join([result_text, 'ㅍ'])
                        elif (letter3[2] == 'ㅀ' or letter3[2] == 'ㄶ'):
                            result_text = "".join([result_text, 'ㅎ'])
                        else:
                            result_text = "".join([result_text, letter3[2]])

                    #    [붙임] 'ㄷ' 뒤에 접미사 '히'가 결합되어 '티'를 이루는 것은 [치]로 발음한다.        
                    elif letter[0] == 'ㅎ' and letter3[2] == 'ㄷ' and letter[2] not in standard_pro_dic:
                        result_text = "".join([result_text, 'ㅊ'])

                    # 제18항 받침 'ㄱ(ㄲ, ㅋ, ㄳ, ㄺ), ㄷ(ㅅ, ㅆ, ㅈ, ㅊ, ㅌ, ㅎ), ㅂ(ㅍ, ㄼ, ㄿ, ㅄ)'은 'ㄴ, ㅁ' 앞에서 [ㅇ, ㄴ, ㅁ]으로 발음한다.
                    # => 기본 발음 그대로 가기

                    # 제19항　받침 'ㅁ, ㅇ' 뒤에 연결되는 'ㄹ'은 [ㄴ]으로 발음한다.
                    elif letter[0] == 'ㄹ' and letter3[2] == 'ㅁ':
                        result_text = "".join([result_text, 'ㄴ'])
                    elif letter[0] == 'ㄹ' and letter3[2] == 'ㅇ':
                        result_text = "".join([result_text, 'ㄴ'])

                    # 제19항 [붙임] 받침 'ㄱ, ㅂ' 뒤에 연결되는 'ㄹ'도 [ㄴ]으로 발음한다.
                    elif letter[0] == 'ㄹ' and (letter3[2] == 'ㄱ' or letter3[2] == 'ㅂ'):
                        result_text = "".join([result_text, 'ㄴ'])

                    # 제23항　받침 'ㄱ(ㄲ, ㅋ, ㄳ, ㄺ), ㄷ(ㅅ, ㅆ, ㅈ, ㅊ, ㅌ), ㅂ(ㅍ, ㄼ, ㄿ, ㅄ)' 뒤에 연결되는 'ㄱ, ㄷ, ㅂ, ㅅ, ㅈ'은 된소리로 발음한다.
                    # 'ㄱ'
                    elif letter[0] == 'ㄱ' and (letter3[2] == 'ㄱ' or letter3[2] == 'ㄲ' or letter3[2] == 'ㅋ' or letter3[2] == 'ㄳ' \
                                                or letter3[2] == 'ㄺ' or letter3[2] == 'ㄷ' or letter3[2] == 'ㅅ' or letter3[2] == 'ㅆ' \
                                                or letter3[2] == 'ㅈ' or letter3[2] == 'ㅊ' or letter3[2] == 'ㅌ' or letter3[2] == 'ㅂ' \
                                                or letter3[2] == 'ㅍ' or letter3[2] == 'ㄼ' or letter3[2] == 'ㄿ' or letter3[2] == 'ㅄ'):
                        result_text = "".join([result_text, 'ㄲ'])
                    # 'ㄷ'  
                    elif letter[0] == 'ㄷ' and (letter3[2] == 'ㄱ' or letter3[2] == 'ㄲ' or letter3[2] == 'ㅋ' or letter3[2] == 'ㄳ' \
                                                or letter3[2] == 'ㄺ' or letter3[2] == 'ㄷ' or letter3[2] == 'ㅅ' or letter3[2] == 'ㅆ' \
                                                or letter3[2] == 'ㅈ' or letter3[2] == 'ㅊ' or letter3[2] == 'ㅌ' or letter3[2] == 'ㅂ' \
                                                or letter3[2] == 'ㅍ' or letter3[2] == 'ㄼ' or letter3[2] == 'ㄿ' or letter3[2] == 'ㅄ'):
                        result_text = "".join([result_text, 'ㄸ'])
                    # 'ㅂ'
                    elif letter[0] == 'ㅂ' and (letter3[2] == 'ㄱ' or letter3[2] == 'ㄲ' or letter3[2] == 'ㅋ' or letter3[2] == 'ㄳ' \
                                                or letter3[2] == 'ㄺ' or letter3[2] == 'ㄷ' or letter3[2] == 'ㅅ' or letter3[2] == 'ㅆ' \
                                                or letter3[2] == 'ㅈ' or letter3[2] == 'ㅊ' or letter3[2] == 'ㅌ' or letter3[2] == 'ㅂ' \
                                                or letter3[2] == 'ㅍ' or letter3[2] == 'ㄼ' or letter3[2] == 'ㄿ' or letter3[2] == 'ㅄ'):
                        result_text = "".join([result_text, 'ㅃ'])
                    # 'ㅅ'
                    elif letter[0] == 'ㅅ' and (letter3[2] == 'ㄱ' or letter3[2] == 'ㄲ' or letter3[2] == 'ㅋ' or letter3[2] == 'ㄳ' \
                                                or letter3[2] == 'ㄺ' or letter3[2] == 'ㄷ' or letter3[2] == 'ㅅ' or letter3[2] == 'ㅆ' \
                                                or letter3[2] == 'ㅈ' or letter3[2] == 'ㅊ' or letter3[2] == 'ㅌ' or letter3[2] == 'ㅂ' \
                                                or letter3[2] == 'ㅍ' or letter3[2] == 'ㄼ' or letter3[2] == 'ㄿ' or letter3[2] == 'ㅄ'):
                        result_text = "".join([result_text, 'ㅆ'])
                    # 'ㅈ'
                    elif letter[0] == 'ㅈ' and (letter3[2] == 'ㄱ' or letter3[2] == 'ㄲ' or letter3[2] == 'ㅋ' or letter3[2] == 'ㄳ' \
                                                or letter3[2] == 'ㄺ' or letter3[2] == 'ㄷ' or letter3[2] == 'ㅅ' or letter3[2] == 'ㅆ' \
                                                or letter3[2] == 'ㅈ' or letter3[2] == 'ㅊ' or letter3[2] == 'ㅌ' or letter3[2] == 'ㅂ' \
                                                or letter3[2] == 'ㅍ' or letter3[2] == 'ㄼ' or letter3[2] == 'ㄿ' or letter3[2] == 'ㅄ'):
                        result_text = "".join([result_text, 'ㅉ'])

                    # 제 9항 받침 'ㄲ, ㅋ', 'ㅅ, ㅆ, ㅈ, ㅊ, ㅌ', 'ㅍ'은 어말 또는 자음 앞에서 각각 대표음 [ㄱ, ㄷ, ㅂ]으로 발음한다.
                    # 예외 키읔과[키윽꽈]
                    elif letter[0] == 'ㄱ' and letter3[2] == 'ㅋ':
                        result_text = "".join([result_text, 'ㄲ'])

                    # 제10항　겹받침 'ㄳ', 'ㄵ', 'ㄼ, ㄽ, ㄾ', 'ㅄ'은 어말 또는 자음 앞에서 각각 [ㄱ, ㄴ, ㄹ, ㅂ]으로 발음한다.
                    # 제11항　겹받침 'ㄺ, ㄻ, ㄿ'은 어말 또는 자음 앞에서 각각 [ㄱ, ㅁ, ㅂ]으로 발음한다.

                    # 제12항 받침 'ㅎ'의 발음은 다음과 같다.
                    # 1. 'ㅎ(ㄶ, ㅀ)' 뒤에 'ㄱ, ㄷ, ㅈ'이 결합되는 경우에는, 뒤 음절 첫소리와 합쳐서 [ㅋ, ㅌ, ㅊ]으로 발음한다.
                    # 'ㅎ'
                    elif letter[0] == 'ㄱ' and letter3[2] == 'ㅎ':
                        result_text = "".join([result_text, 'ㅋ'])
                    elif letter[0] == 'ㄷ' and letter3[2] == 'ㅎ':
                        result_text = "".join([result_text, 'ㅌ'])
                    elif letter[0] == 'ㅈ' and letter3[2] == 'ㅎ':
                        result_text = "".join([result_text, 'ㅊ'])
                    # 'ㄶ'
                    elif letter[0] == 'ㄱ' and letter3[2] == 'ㄶ':
                        result_text = "".join([result_text, 'ㅋ'])
                    elif letter[0] == 'ㄷ' and letter3[2] == 'ㄶ':
                        result_text = "".join([result_text, 'ㅌ'])
                    elif letter[0] == 'ㅈ' and letter3[2] == 'ㄶ':
                        result_text = "".join([result_text, 'ㅊ'])
                    # 'ㅀ'    
                    elif letter[0] == 'ㄱ' and letter3[2] == 'ㅀ':
                        result_text = "".join([result_text, 'ㅋ'])
                    elif letter[0] == 'ㄷ' and letter3[2] == 'ㅀ':
                        result_text = "".join([result_text, 'ㅌ'])
                    elif letter[0] == 'ㅈ' and letter3[2] == 'ㅀ':
                        result_text = "".join([result_text, 'ㅊ'])
                    #     [붙임 1] 받침 'ㄱ(ㄺ),ㄷ,ㅂ(ㄼ),ㅈ(ㄵ)'이 뒤 음절 첫소리 'ㅎ'과 결합되는 경우에도,
                    #              역시 두 소리를 합쳐서 [ㅋ,ㅌ,ㅍ,ㅊ]으로 발음한다.
                    # 'ㄱ,ㄷ,ㅂ,ㅈ'
                    elif letter[0] == 'ㅎ' and letter3[2] == 'ㄱ':
                        result_text = "".join([result_text, 'ㅋ'])
                    elif letter[0] == 'ㅎ' and letter3[2] == 'ㄷ':
                        result_text = "".join([result_text, 'ㅌ'])
                    elif letter[0] == 'ㅎ' and letter3[2] == 'ㅂ':
                        result_text = "".join([result_text, 'ㅍ'])
                    elif letter[0] == 'ㅎ' and letter3[2] == 'ㅈ':
                        result_text = "".join([result_text, 'ㅊ'])
                    # 'ㄺ,ㄼ,ㄵ'
                    elif letter[0] == 'ㅎ' and letter3[2] == 'ㄺ':
                        result_text = "".join([result_text, 'ㅋ'])
                    elif letter[0] == 'ㅎ' and letter3[2] == 'ㄼ':
                        result_text = "".join([result_text, 'ㅍ'])
                    elif letter[0] == 'ㅎ' and letter3[2] == 'ㄵ':
                        result_text = "".join([result_text, 'ㅊ'])
                    #     [붙임 2] 규정에 따라 'ㄷ'으로 발음되는 'ㅅ, ㅈ, ㅊ, ㅌ'의 경우에는 이에 준한다.
                    # 'ㅅ,ㅈ,ㅊ,ㅌ'
                    elif letter[0] == 'ㅎ' and (letter3[2] == 'ㅅ' or letter3[2] == 'ㅈ' or letter3[2] == 'ㅊ' or letter3[2] == 'ㅌ'):
                        result_text = "".join([result_text, 'ㅌ'])
                    #    2. 'ㅎ(ㄶ, ㅀ)' 뒤에 'ㅅ'이 결합되는 경우에는, 'ㅅ'을 [ㅆ]으로 발음한다.
                    elif letter[0] == 'ㅅ' and (letter3[2] == 'ㅎ' or letter3[2] == 'ㄶ' or letter3[2] == 'ㅀ'):
                        result_text = "".join([result_text, 'ㅆ'])
                    #    3. 'ㅎ' 뒤에 'ㄴ'이 결합되는 경우에는, [ㄴ]으로 발음한다.
                    #     [붙임] 'ㄶ, ㅀ' 뒤에 'ㄴ'이 결합되는 경우에는, 'ㅎ'을 발음하지 않는다.
                    # 'ㄶ, ㅀ'
                    elif letter[0] == 'ㄴ' and letter3[2] == 'ㄶ':
                        result_text = "".join([result_text, 'ㄴ'])
                    elif letter[0] == 'ㄴ' and letter3[2] == 'ㅀ':
                        result_text = "".join([result_text, 'ㄹ'])

                    # 제25항 어간 받침 'ㄼ, ㄾ' 뒤에 결합되는 어미의 첫소리 'ㄱ, ㄷ, ㅅ, ㅈ'은 된소리로 발음한다.
                    elif letter[0] == 'ㄱ' and (letter3[2] == 'ㄼ' or letter3[2] == 'ㄾ'):
                        result_text = "".join([result_text, 'ㄲ'])
                    elif letter[0] == 'ㄷ' and (letter3[2] == 'ㄼ' or letter3[2] == 'ㄾ'):
                        result_text = "".join([result_text, 'ㄸ'])
                    elif letter[0] == 'ㅅ' and (letter3[2] == 'ㄼ' or letter3[2] == 'ㄾ'):
                        result_text = "".join([result_text, 'ㅆ'])
                    elif letter[0] == 'ㅈ' and (letter3[2] == 'ㄼ' or letter3[2] == 'ㄾ'):
                        result_text = "".join([result_text, 'ㅉ'])

                    # 제27항　관형사형 '-[으]ㄹ' 뒤에 연결되는 'ㄱ, ㄷ, ㅂ, ㅅ, ㅈ'은 된소리로 발음한다.
                    # 할~ 갈~ 날~
                    elif (letter3[0] == 'ㅎ' or letter3[0] == 'ㄱ' or letter3[0] == 'ㄴ') and letter3[1] == 'ㅏ' and letter3[2] == 'ㄹ' and letter[0] == 'ㄱ':
                        result_text = "".join([result_text, 'ㄲ'])
                    elif (letter3[0] == 'ㅎ' or letter3[0] == 'ㄱ' or letter3[0] == 'ㄴ') and letter3[1] == 'ㅏ' and letter3[2] == 'ㄹ' and letter[0] == 'ㄷ':
                        result_text = "".join([result_text, 'ㄸ'])
                    elif (letter3[0] == 'ㅎ' or letter3[0] == 'ㄴ') and letter3[1] == 'ㅏ' and letter3[2] == 'ㄹ' and letter[0] == 'ㅂ':
                        result_text = "".join([result_text, 'ㅃ'])
                    # 소갈비[소갈비]인데 [소갈삐]로 출력됨
                    elif letter3[0] == 'ㄱ' and letter3[1] == 'ㅏ' and letter3[2] == 'ㄹ' and letter[0] == 'ㅂ':
                        if letter[1] == 'ㅣ' and letter[2] not in standard_pro_dic:
                            result_text = "".join([result_text, 'ㅂ'])
                        else:
                            result_text = "".join([result_text, 'ㅃ'])
                    elif (letter3[0] == 'ㅎ' or letter3[0] == 'ㄱ' or letter3[0] == 'ㄴ') \
                    and letter3[1] == 'ㅏ' and letter3[2] == 'ㄹ' and letter[0] == 'ㅅ':
                        result_text = "".join([result_text, 'ㅆ'])
                    elif (letter3[0] == 'ㅎ' or letter3[0] == 'ㄱ' or letter3[0] == 'ㄴ') \
                    and letter3[1] == 'ㅏ' and letter3[2] == 'ㄹ' and letter[0] == 'ㅈ':
                        result_text = "".join([result_text, 'ㅉ'])
                    else:
                        result_text = "".join([result_text, kata_standard_pro['standard_pro_JA'][np.where(letter[0]==standard_pro_dic)[0][0]]])

                if letter[1] in standard_pro_dic:
                    result_text = "".join([result_text, kata_standard_pro['standard_pro_JA'][np.where(letter[1]==standard_pro_dic)[0][0]]])

                if letter[2] in standard_pro_dic:
                    if i == (len(text)-1):
                        result_text = "".join([result_text, kata_standard_pro['standard_pro_JONG'][np.where(letter[2]==standard_pro_dic)[0][0]]])
                    if i != (len(text)-1):
                        # 한글 + 한글이외의 문자가 결합될 시 exception 오류가 발견되어
                        # 치킨롤&포테이포 => 치킨로롤&포테이토 라고 출력나옴
                        try:
                            letter2 = hgtk.letter.decompose(text[i+1])
                            if letter2[0] == 'ㅇ' and letter[2] in standard_pro_dic:
                                
                                # 깻잎 [깨십] -> [깬닙]
                                # 사이시옷 뒤에 ‘이’ 음이 결합되는 경우에는 [ㄴㄴ]으로 발음한다
                                if letter[2] == 'ㅅ':
                                    result_text = "".join([result_text, 'ㄴ'])
                                elif (letter[2] == 'ㄷ' or letter[2] == 'ㅌ') and letter2[1] == 'ㅣ':
                                    result_text = "".join([result_text, '▒'])
                                elif letter[2] in standard_pro_dic and letter[2] not in 'ㄳ,ㄵ,ㄼ,ㄽ,ㄾ,ㅄ,ㄺ,ㄻ,ㄿ,ㅀ,ㄶ,ㅃ,ㅉ,ㄸ,ㅎ' and letter2[1] in standard_pro_dic:
                                    if letter[2] == 'ㅇ':
                                        result_text = "".join([result_text, 'ㅇ'])
                                    else:
                                        result_text = "".join([result_text, '▒'])
                                elif letter[2] in 'ㄳ,ㄵ,ㄼ,ㄽ,ㄾ,ㅄ,ㄺ,ㄻ,ㄿ,ㅀ,ㄶ' and letter2[1] in standard_pro_dic:
                                    if letter[2] == 'ㄳ' and letter2[0] == 'ㅇ':
                                        result_text = "".join([result_text, 'ㄱ'])
                                    elif (letter[2] == 'ㄵ' or letter[2] == 'ㄶ') and letter2[0] == 'ㅇ':
                                        result_text = "".join([result_text, 'ㄴ'])
                                    elif letter[2] == 'ㅄ' and letter2[0] == 'ㅇ':
                                        result_text = "".join([result_text, 'ㅂ'])
                                    elif (letter[2] == 'ㄺ' or letter[2] == 'ㄼ' or letter[2] == 'ㄽ' \
                                         or letter[2] == 'ㄻ' or letter[2] == 'ㄾ' or letter[2] == 'ㅀ') and letter2[0] == 'ㅇ':
                                        result_text = "".join([result_text, 'ㄹ'])
                                    # 종성'ㅇ', 초성'ㅇ'인 경우 종성'ㅇ'이 사라집니다. 예) 능이->느이 , 장어->자어 , 영양 -> 여양
                                    elif letter[2] == 'ㅇ' and letter2[0] == 'ㅇ':
                                        result_text = "".join([result_text, 'ㅇ'])

                                else:
                                    result_text = "".join([result_text, kata_standard_pro['standard_pro_MO_JONG'][np.where(letter[2]==standard_pro_dic)[0][0]]])

                            # 제10항　겹받침 'ㄳ', 'ㄵ', 'ㄼ, ㄽ, ㄾ', 'ㅄ'은 어말 또는 자음 앞에서 각각 [ㄱ, ㄴ, ㄹ, ㅂ]으로 발음한다.
                            # 다만, '밟-'은 자음 앞에서 [밥]으로 발음하고, '넓-'은 다음과 같은 경우에 [넙]으로 발음한다.
                            elif letter[0] == 'ㅂ' and letter[1] == 'ㅏ' and letter[2] == 'ㄼ' and letter2[0] != 'ㄴ' and letter2[0] in standard_pro_dic:
                                result_text = "".join([result_text, 'ㅂ'])
                            elif letter[0] == 'ㅂ' and letter[1] == 'ㅏ' and letter[2] == 'ㄼ' and letter2[0] == 'ㄴ':
                                result_text = "".join([result_text, 'ㅁ'])
                            elif letter[0] == 'ㄴ' and letter[1] == 'ㅓ' and letter[2] == 'ㄼ' and letter2[0] in standard_pro_dic:
                                result_text = "".join([result_text, 'ㅂ'])

                            # 제11항　겹받침 'ㄺ, ㄻ, ㄿ'은 어말 또는 자음 앞에서 각각 [ㄱ, ㅁ, ㅂ]으로 발음한다.
                            # 제11항 다만 예외 중 흙과[흑꽈]에 관련된 내용
                            elif letter[2] == 'ㄺ' and letter2[0] == 'ㄱ' and letter2[1] in 'ㅘ,ㅟ,ㅙ,ㅝ,ㅞ':
                                result_text = "".join([result_text, 'ㄱ'])

                            # 다만, 용언의 어간 말음 'ㄺ'은 'ㄱ' 앞에서 [ㄹ]로 발음한다.
                            elif letter[2] == 'ㄺ' and letter2[0] == 'ㄱ' and letter2[1] not in 'ㅘ,ㅟ,ㅙ,ㅝ,ㅞ':
                                result_text = "".join([result_text, 'ㄹ'])

                            # 넓죽하다 -> 넙쭈카다
                            # 제12항 받침 'ㅎ'의 발음은 다음과 같다.
                            #    1. 'ㅎ(ㄶ, ㅀ)' 뒤에 'ㄱ, ㄷ, ㅈ'이 결합되는 경우에는, 뒤 음절 첫소리와 합쳐서 [ㅋ, ㅌ, ㅊ]으로 발음한다.
                            # 'ㅎ'
                            elif letter[2] == 'ㅎ' and (letter2[0] == 'ㄱ' or letter2[0] == 'ㄷ' or letter2[0] == 'ㅈ'):
                                result_text = "".join([result_text, '▒'])
                            # 'ㄶ'
                            elif letter[2] == 'ㄶ' and (letter2[0] == 'ㄱ' or letter2[0] == 'ㄷ' or letter2[0] == 'ㅈ'):
                                result_text = "".join([result_text, 'ㄴ'])
                            # 'ㅀ'
                            elif letter[2] == 'ㅀ' and (letter2[0] == 'ㄱ' or letter2[0] == 'ㄷ' or letter2[0] == 'ㅈ'):
                                result_text = "".join([result_text, 'ㄹ'])
                            #     [붙임 1] 받침 'ㄱ(ㄺ),ㄷ,ㅂ(ㄼ),ㅈ(ㄵ)'이 뒤 음절 첫소리 'ㅎ'과 결합되는 경우에도,
                            #              역시 두 소리를 합쳐서 [ㅋ,ㅌ,ㅍ,ㅊ]으로 발음한다.
                            # 'ㄱ,ㄷ,ㅂ,ㅈ'
                            elif (letter[2] == 'ㄱ' or letter[2] == 'ㄷ' or letter[2] == 'ㅂ' or letter[2] == 'ㅈ') and letter2[0] == 'ㅎ':
                                result_text = "".join([result_text, '▒'])
                            # 'ㄺ,ㄼ,ㄵ'
                            elif letter[2] == 'ㄺ' and letter2[0] == 'ㅎ':
                                result_text = "".join([result_text, 'ㄱ'])
                            elif letter[2] == 'ㄼ' and letter2[0] == 'ㅎ':
                                result_text = "".join([result_text, 'ㄹ'])
                            elif letter[2] == 'ㄵ' and letter2[0] == 'ㅎ':
                                result_text = "".join([result_text, 'ㄴ'])
                            #     [붙임 2] 규정에 따라 'ㄷ'으로 발음되는 'ㅅ, ㅈ, ㅊ, ㅌ'의 경우에는 이에 준한다.
                            # 'ㅅ,ㅈ,ㅊ,ㅌ'
                            elif (letter[2] == 'ㅅ' or letter[2] == 'ㅈ' or letter[2] == 'ㅊ' or letter[2] == 'ㅌ') and letter2[0] == 'ㅎ':
                                result_text = "".join([result_text, '▒'])

                            #    2. 'ㅎ(ㄶ, ㅀ)' 뒤에 'ㅅ'이 결합되는 경우에는, 'ㅅ'을 [ㅆ]으로 발음한다.
                            elif letter[2] == 'ㅎ' and letter2[0] == 'ㅅ':
                                result_text = "".join([result_text, '▒'])
                            elif letter[2] == 'ㄶ' and letter2[0] == 'ㅅ':
                                result_text = "".join([result_text, 'ㄴ'])
                            elif letter[2] == 'ㅀ' and letter2[0] == 'ㅅ':
                                result_text = "".join([result_text, 'ㄹ'])

                            #    3. 'ㅎ' 뒤에 'ㄴ'이 결합되는 경우에는, [ㄴ]으로 발음한다.
                            elif letter[2] == 'ㅎ' and letter2[0] == 'ㄴ':
                                result_text = "".join([result_text, 'ㄴ'])
                            #     [붙임] 'ㄶ, ㅀ' 뒤에 'ㄴ'이 결합되는 경우에는, 'ㅎ'을 발음하지 않는다.
                            # 'ㄶ, ㅀ'
                            elif letter[2] == 'ㄶ' and letter2[0] == 'ㄴ':
                                result_text = "".join([result_text, 'ㄴ'])
                            elif letter[2] == 'ㅀ' and letter2[0] == 'ㄴ':
                                result_text = "".join([result_text, 'ㄹ'])

                            #    4. 'ㅎ(ㄶ, ㅀ)' 뒤에 모음으로 시작된 어미나 접미사가 결합되는 경우에는, 'ㅎ'을 발음하지 않는다.
                            elif (letter[2] == 'ㅎ' or letter[2] == 'ㄶ' or letter[2] == 'ㅀ') and letter2[0] == 'ㅇ':
                                result_text = "".join([result_text, '▒'])

                            #    [붙임] 'ㄷ' 뒤에 접미사 '히'가 결합되어 '티'를 이루는 것은 [치]로 발음한다.
                            elif letter2[1] == 'l' and letter[2] == 'ㄷ' and letter2[0] == 'ㅎ' and letter2[2] not in standard_pro_dic:
                                result_text = "".join([result_text, '▒'])

                            # 제13항 홑받침이나 쌍받침이 모음으로 시작된 조사나 어미, 접미사와 결합되는 경우에는, 제 음가대로 뒤 음절 첫소리로 옮겨 발음한다.
                            elif letter2[0] == 'ㅇ' and letter[2] in standard_pro_dic and letter[2] not in 'ㄳ,ㄵ,ㄼ,ㄽ,ㄾ,ㅄ,ㄺ,ㄻ,ㄿ,ㅀ,ㄶ,ㅃ,ㅉ,ㄸ':
                                # 종성'ㅇ', 초성'ㅇ'인 경우 종성'ㅇ'이 사라집니다. 예) 능이->느이 , 장어->자어 , 영양 -> 여양
                                if letter[2] == 'ㅇ' and letter2[0] == 'ㅇ':
                                    result_text = "".join([result_text, 'ㅇ'])
                                else:
                                    result_text = "".join([result_text, '▒'])

                            # 제14항　겹받침이 모음으로 시작된 조사나 어미, 접미사와 결합되는 경우에는, 뒤엣것만을 뒤 음절 첫소리로 옮겨 발음한다.
                            # (이 경우, 'ㅅ'은 된소리로 발음함.)    
                            elif letter2[0] == 'ㅇ' and letter[2] in 'ㄳ,ㄵ,ㄼ,ㄽ,ㄾ,ㅄ,ㄺ,ㄻ,ㄿ,ㅀ,ㄶ':
                                if letter[2] == 'ㄳ' and letter2[0] == 'ㅇ':
                                    result_text = "".join([result_text, 'ㄱ'])
                                elif (letter[2] == 'ㄵ' or letter[2] == 'ㄶ') and letter2[0] == 'ㅇ':
                                    result_text = "".join([result_text, 'ㄴ'])
                                elif letter[2] == 'ㅄ' and letter2[0] == 'ㅇ':
                                    result_text = "".join([result_text, 'ㅂ'])
                                elif (letter[2] == 'ㄺ' or letter[2] == 'ㄻ' or letter[2] == 'ㄼ' or letter[2] == 'ㄽ' \
                                     or letter[2] == 'ㄾ' or letter[2] == 'ㄿ' or letter[2] == 'ㅀ') and letter2[0] == 'ㅇ':
                                    result_text = "".join([result_text, 'ㄹ'])
                                # 종성'ㅇ', 초성'ㅇ'인 경우 종성'ㅇ'이 사라집니다. 예) 능이->느이 , 장어->자어 , 영양 -> 여양
                                elif letter[2] == 'ㅇ' and letter2[0] == 'ㅇ':
                                    result_text = "".join([result_text, 'ㅇ'])
                                else:
                                    result_text = "".join([result_text, '▒'])

                            # 제18항 받침 'ㄱ(ㄲ, ㅋ, ㄳ, ㄺ), ㄷ(ㅅ, ㅆ, ㅈ, ㅊ, ㅌ, ㅎ), ㅂ(ㅍ, ㄼ, ㄿ, ㅄ)'은 'ㄴ, ㅁ' 앞에서 [ㅇ, ㄴ, ㅁ]으로 발음한다.
                            # 'ㄱ,ㄲ,ㅋ,ㄳ,ㄺ' -> 'ㅇ'
                            elif (letter[2] == 'ㄱ' or letter[2] == 'ㄲ' or letter[2] == 'ㅋ' or letter[2] == 'ㄳ' or letter[2] == 'ㄺ') \
                            and (letter2[0] == 'ㄴ' or letter2[0] == 'ㅁ'):
                                result_text = "".join([result_text, 'ㅇ'])
                            # 'ㄷ.ㅅ.ㅆ.ㅈ.ㅊ.ㅌ.ㅎ' -> 'ㄴ'
                            elif (letter[2] == 'ㄷ' or letter[2] == 'ㅅ' or letter[2] == 'ㅆ' or letter[2] == 'ㅈ'  \
                                  or letter[2] == 'ㅊ' or letter[2] == 'ㅌ' or letter[2] == 'ㅎ') \
                            and (letter2[0] == 'ㄴ' or letter2[0] == 'ㅁ'):
                                result_text = "".join([result_text, 'ㄴ'])
                            # 'ㅂ,ㅍ,ㄼ,ㄿ,ㅄ' -> 'ㅁ'
                            elif (letter2[2] == 'ㅂ' or letter[2] == 'ㅍ' or letter[2] == 'ㄼ' or letter[2] == 'ㄿ' or letter[2] == 'ㅄ') \
                            and (letter2[0] == 'ㄴ' or letter2[0] == 'ㅁ'):
                                result_text = "".join([result_text, 'ㅁ'])

                            # 제19항 [붙임] 받침 'ㄱ, ㅂ' 뒤에 연결되는 'ㄹ'도 [ㄴ]으로 발음한다.
                            elif letter[2] == 'ㄱ' and letter2[0] == 'ㄹ':
                                result_text = "".join([result_text, 'ㅇ'])
                            elif letter[2] == 'ㅂ' and letter2[0] == 'ㄹ':
                                result_text = "".join([result_text, 'ㅁ'])

                            # 제23항 받침 'ㄱ(ㄲ, ㅋ, ㄳ, ㄺ), ㄷ(ㅅ, ㅆ, ㅈ, ㅊ, ㅌ), ㅂ(ㅍ, ㄼ, ㄿ, ㅄ)' 뒤에 연결되는 'ㄱ, ㄷ, ㅂ, ㅅ, ㅈ'은 된소리로 발음한다.
                            # => 종성은 변화규칙X 초성만 변화규칙 적용
                            elif letter2[0] == 'ㅇ' and letter[2] not in standard_pro_dic:
                                result_text = "".join([result_text, letter[2]])
                            elif letter2[0] != 'ㅇ' and letter[2] in standard_pro_dic:
                                result_text = "".join([result_text, kata_standard_pro['standard_pro_JONG'][np.where(letter[2]==standard_pro_dic)[0][0]]])

                        except:
                            result_text = "".join([result_text, kata_standard_pro['standard_pro_JONG'][np.where(letter[2]==standard_pro_dic)[0][0]]])

        except:
            result_text = "".join([result_text, text[i]])

    result = join_jamos(result_text)
    result = str(result).replace('▒','')
    return result

def auto_katakanize(input_text):
    # 문자가 아니면 그대로 출력
    if type(input_text) != str:
        final_result = input_text
        return final_result

    if type(input_text) == str:
        if isHangul(input_text) == False:
            final_result = input_text
            return final_result

    # 관용어사전 적용
    input_text = kata_idiom_replace(input_text)

    while len(re.compile('[가-힣]+').findall(input_text)) > 0:
        # 한글만 추출
        one_text = re.compile('[가-힣]+').findall(input_text)
        for i in range(len(one_text)):
            # 표준발음법
            co_text = standardPro(one_text[i])
            # 종성 분리
            co_text = segment(co_text)
            # 받침 예외처리
            co_text = JONG_exception(co_text)
            # 자모음 예외처리1 유성음
            co_text = JAMO_exception(co_text)
            # 자모음 예외처리2 
            co_text = JAMO_exception2(co_text)
            # 가타카나 변환
            co_text = katakanize(co_text)
            input_text = input_text.replace(one_text[i], co_text)

    final_result = input_text
    return final_result

In [2]:
auto_katakanize("도리깻열")

'トリケンニョル'

In [4]:
standardPro("안녕하세요")

'안녕하세요'

In [4]:
file_path = "C:/Users/opjet/Downloads/"

df = pd.read_excel(file_path + "[빅데이터팀] 지하철역 번역_211118.xlsx", sheet_name="지하철추가_20231027")

In [5]:
df

,운영기관명,노선명,역번호,역사명,영문역사명,한자역사명,역사도로명주소,EN 역사명,JP 역사명,CN 역사명,TW 역사명,지번주소,도로명주소
0,대구교통공사,1호선,123.0,서부정류장\n (관문시장),Seobu Bus Terminal\n (Gwanmun Market),西部客運站\n （關門市場）,남구 월배로 지하501 (대명동),Seobu Bus Terminal (Gwanmun Market),ソブジョンニュジャン駅,西部客运站,西部客運站,대구광역시 남구 대명동 1576-21,대구광역시 남구 월배로 지하 501
1,대구교통공사,1호선,136.0,동구청\n (큰고개),Dong-gu Office\n (Keungogae),東區廳,동구 신암남로 지하169 (신암동),Dong-gu Office (Keungogae),NaN,东区厅站,東區廳站,대구광역시 동구 신암동 369,대구광역시 동구 신암남로 지하 169
2,대구교통공사,2호선,229.0,청라언덕,Cheongna Hill,靑蘿坡,중구 달구벌대로 지하2000 (동산동),Cheongna Hill,NaN,青萝坡站,青蘿坡站,대구광역시 중구 동산동 640,대구광역시 중구 달구벌대로 지하 2000
3,대구교통공사,3호선,329.0,청라언덕\n (신남),Cheongna Hill\n (sinnam),靑蘿坡\n (新南),중구 남산로 80(남산동),Cheongna Hill (sinnam),NaN,青萝坡站,青蘿坡站,대구광역시 중구 남산동 97-1,대구광역시 중구 남산로 80
4,대구교통공사,3호선,336.0,어린이세상,Children's World,儿童世界,수성구 동대구로 166(황금동),Children's World,NaN,儿童世界站,兒童世界站,대구광역시 수성구 황금동 908,대구광역시 수성구 동대구로 166
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,한국철도공사,서해,4810.0,시흥능곡역,Siheung Neunggok,始興陵谷,경기도 시흥시 시흥대로 지하 327,Siheung Neunggok,NaN,始兴陵谷站,始興陵谷站,경기도 시흥시 능곡동 717,경기도 시흥시 시흥대로 지하 327
78,한국철도공사,서해,4811.0,달미역,Dalmi,달미,경기도 안산시 단원구 순환로 지하 160,Dalmi,NaN,达美站,達美站,경기도 안산시 단원구 선부동 481,경기도 안산시 단원구 순환로 지하 160
79,한국철도공사,서해,4812.0,선부역,Seonbu,仙府,경기도 안산시 단원구 선부광장로 34,Seonbu,NaN,仙府站,仙府站,경기도 안산시 단원구 선부동 1076-16,경기도 안산시 단원구 선부광장로 33
80,한국철도공사,서해,4814.0,시우역,Siu,時雨,경기도 안산시 단원구 동산로 지하 50,Siu,NaN,时雨站,時雨站,경기도 안산시 단원구 원시동 804,경기도 안산시 단원구 동산로 지하 50


In [17]:
import re

pattern = r'\([^)]*\)'
result = []

for i in range(len(df["운영기관명"])):
    subway_nm = df["역사명"][i]
    subway_nm = subway_nm.replace("\n", "")
    subway_nm = re.sub(pattern=pattern, repl='', string=subway_nm)
    
    if subway_nm.startswith(" "):
        subway_nm = subway_nm[1:]
    if subway_nm.endswith(" "):
        subway_nm = subway_nm[:-1]
    if subway_nm.endswith("역"):
        subway_nm = subway_nm[:-1]
        
    result.append(auto_katakanize(subway_nm) + "駅")
    
df["JP"] = result

In [13]:
sample_text = " 안녕~"

In [14]:
sample_text[1:]

'안녕~'

In [15]:
sample_text[:-1]

' 안녕'

In [18]:
df

,운영기관명,노선명,역번호,역사명,영문역사명,한자역사명,역사도로명주소,EN 역사명,JP 역사명,CN 역사명,TW 역사명,지번주소,도로명주소,JP
0,대구교통공사,1호선,123.0,서부정류장\n (관문시장),Seobu Bus Terminal\n (Gwanmun Market),西部客運站\n （關門市場）,남구 월배로 지하501 (대명동),Seobu Bus Terminal (Gwanmun Market),ソブジョンニュジャン駅,西部客运站,西部客運站,대구광역시 남구 대명동 1576-21,대구광역시 남구 월배로 지하 501,ソブジョンニュジャン駅
1,대구교통공사,1호선,136.0,동구청\n (큰고개),Dong-gu Office\n (Keungogae),東區廳,동구 신암남로 지하169 (신암동),Dong-gu Office (Keungogae),NaN,东区厅站,東區廳站,대구광역시 동구 신암동 369,대구광역시 동구 신암남로 지하 169,トングチョン駅
2,대구교통공사,2호선,229.0,청라언덕,Cheongna Hill,靑蘿坡,중구 달구벌대로 지하2000 (동산동),Cheongna Hill,NaN,青萝坡站,青蘿坡站,대구광역시 중구 동산동 640,대구광역시 중구 달구벌대로 지하 2000,チョンナオンドク駅
3,대구교통공사,3호선,329.0,청라언덕\n (신남),Cheongna Hill\n (sinnam),靑蘿坡\n (新南),중구 남산로 80(남산동),Cheongna Hill (sinnam),NaN,青萝坡站,青蘿坡站,대구광역시 중구 남산동 97-1,대구광역시 중구 남산로 80,チョンナオンドク駅
4,대구교통공사,3호선,336.0,어린이세상,Children's World,儿童世界,수성구 동대구로 166(황금동),Children's World,NaN,儿童世界站,兒童世界站,대구광역시 수성구 황금동 908,대구광역시 수성구 동대구로 166,オリニセサン駅
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,한국철도공사,서해,4810.0,시흥능곡역,Siheung Neunggok,始興陵谷,경기도 시흥시 시흥대로 지하 327,Siheung Neunggok,NaN,始兴陵谷站,始興陵谷站,경기도 시흥시 능곡동 717,경기도 시흥시 시흥대로 지하 327,シフンヌンゴク駅
78,한국철도공사,서해,4811.0,달미역,Dalmi,달미,경기도 안산시 단원구 순환로 지하 160,Dalmi,NaN,达美站,達美站,경기도 안산시 단원구 선부동 481,경기도 안산시 단원구 순환로 지하 160,タルミ駅
79,한국철도공사,서해,4812.0,선부역,Seonbu,仙府,경기도 안산시 단원구 선부광장로 34,Seonbu,NaN,仙府站,仙府站,경기도 안산시 단원구 선부동 1076-16,경기도 안산시 단원구 선부광장로 33,ソンブ駅
80,한국철도공사,서해,4814.0,시우역,Siu,時雨,경기도 안산시 단원구 동산로 지하 50,Siu,NaN,时雨站,時雨站,경기도 안산시 단원구 원시동 804,경기도 안산시 단원구 동산로 지하 50,シウ駅


In [20]:
df.to_excel(file_path + "확인2.xlsx", index=False)